In [ ]:
########## Settings ##########
import os
import sys
from dotenv import load_dotenv

def GetRoot() -> None:
    """
    Set the root directory of the project.
    Usage in Python script:
    GetRoot()
    root = os.environ['PROJECT_ROOT']
    :return:None
    """
    # Allow to use the ".env" file
    load_dotenv()

    abs_path = os.path.abspath(os.getcwd())
    project_root = os.path.dirname(abs_path)
    os.environ['PROJECT_ROOT'] = project_root
    return None

def GetModule() -> None:
    # get the root directory of this project
    GetRoot()

    # get and set the environment variables
    python_path = os.environ.get('PYTHONPATH')
    if python_path:
        for path in python_path.split(':'):
            path = str(os.environ['PROJECT_ROOT']) + path
            if path not in sys.path:
                sys.path.append(path)
    return None

# Get the project root and load environment variables
GetModule()

########## Import Packages ##########
import numpy as  np
import pandas as pd
import tensorflow as tf
from keras.utils import to_categorical
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import plot_model
import matplotlib.pyplot as plt
import time

# Set the random seed
np.random.seed(7)

# Show all contents in a np.array
np.set_printoptions(threshold=np.inf)

In [ ]:
# Self-defining Functions
# Build a function to draw the training data
def plot_img_and_lab(imgs, labels, prediction, idx, num=10):
    # Set the image size
    fig = plt.gcf()
    fig.set_size_inches(12, 14)

    # 25 images the most
    if num>25:num=25

    for i in range(0, num):
        # Build sub-plots (5 rows * 5 columns)
        ax = plt.subplot(5, 5, i+1)

        # Draw sub-plots
        ax.imshow(imgs[idx], cmap="binary")

        # Add the title and label
        title = f"label: {str(labels[idx])}"

        # Show the prediction result if so
        if len(prediction) >0:
            title += f", prediction: {str(prediction[idx])}"
        
        # Set the title size of sub-plots
        ax.set_title(title, fontsize = 20)

        # Not show the ticks
        ax.set_xticks([]);ax.set_yticks([])
        idx += 1
    plt.show()

# Show train history
def show_train_history(train_history, train, validation, path):
    # Plotting
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title("Train history")
    plt.ylabel("train")
    plt.xlabel("epoch")

    # Set legend
    plt.legend(["train", "validation"], loc="upper left")
    plt.savefig(path)
    plt.show()

In [ ]:
# Import MNIST dataset
(x_train_img, y_train_lab), (x_test_img, y_test_lab) = mnist.load_data()
print(f"Train data: {x_train_img.shape[0]}; Train data size: {x_train_img.shape[1]}")
print(f"Test data: {x_test_img.shape[0]}; Test data size: {x_test_img.shape[1]}")

# Plot the first 10 MNIST images
plot_img_and_lab(x_train_img, y_train_lab, [], 0, num=10)

# Show the example image
print(f"The first image label: {y_train_lab[0]}")
plt.imshow(x_train_img[0], cmap="binary", vmin=0, vmax=255)
plt.show()

# Numerical image
x_train_img = x_train_img.reshape(x_train_img.shape[0], 28*28).astype("float32")
x_test_img = x_test_img.reshape(x_test_img.shape[0], 28*28).astype("float32")
# print(f"The numerical image :\n{x_train_img[0]}")

# Normalized image
x_train_img = x_train_img/255
x_test_img = x_test_img/255
# print(f"The normalized image :\n{x_train_img[0]}")

# Label processing
y_train_lab_cat = to_categorical(y_train_lab)
y_test_lab_cat = to_categorical(y_test_lab)
# print(f"The categorical label :\n{y_train_lab[0]}")

In [ ]:
# Build a DNN model
model = Sequential()
model.add(Dense(256, input_dim=28*28, activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(10, activation="softmax"))
model.summary()

# Visualize the model
plot_model(
    model,
    to_file="../docs/mnist_dnn_model_01.png",
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
    dpi=100,
    show_layer_activations=True,
    show_trainable=True)

In [ ]:
# Training
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
train_history = model.fit(x_train_img, y_train_lab_cat, epochs=10, batch_size=128, shuffle=True, validation_split=0.1, verbose=1)
loss, accuracy = model.evaluate(x_test_img, y_test_lab_cat)
print(f"Accuracy: {accuracy}; Loss: {loss}")

In [ ]:
# Draw and save images of the training process
# Create the saving folder for images and the model
dir_result = os.getenv("PROJECT_ROOT") + os.getenv("DIR_RESULTS") + "01_mnist_dnn/"
dir_model = os.getenv("PROJECT_ROOT") + os.getenv("DIR_MODELS") + "01_mnist_dnn/"
os.makedirs(dir_result, exist_ok=True)
os.makedirs(dir_model, exist_ok=True)

# Save results and the model
time_tag = time.strftime("%Y%m%d_%H%M%S", time.localtime())
fil_acc_img = os.path.join(dir_result, f"train_val_accuracy_{time_tag}.png")
fil_loss_img = os.path.join(dir_result, f"train_val_loss_{time_tag}.png")
show_train_history(train_history, "accuracy", "val_accuracy", fil_acc_img)
show_train_history(train_history, "loss", "val_loss", fil_loss_img)

# Save model
fil_model = os.path.join(dir_model, f"mnist_dnn_training_{time_tag}.keras")
model.save(fil_model)

In [ ]:
# Independent test
score = model.evaluate(x_test_img, y_test_lab_cat, verbose=1)
print("===== Independent test =====")
print(f"Test accuracy: {score[1]}")
print(f"Test loss: {score[0]}")

# Prediction
results = model.predict(x_test_img[:10])
print("\n===== Prediction =====")
print(f"The first 10 true labels: {y_test_lab[:10].astype(int)}")
print(f"The first 10 prediction results: {np.argmax(results, axis=-1)}")

# Independent test (reload the model by the saving path)
model_path = r"../models/01_mnist_dnn/mnist_dnn_training_20250224_205014.keras"
model_reload = tf.keras.models.load_model(model_path)
score_reload = model_reload.evaluate(x_test_img, y_test_lab_cat, verbose=1)
print("\n===== Independent test (reloading the model) =====")
print(f"Test accuracy: {score_reload[1]}")
print(f"Test loss: {score_reload[0]}")

# Prediction (reload the model by the saving path)
results_reload = model_reload.predict(x_test_img[:10])
print("\n===== Prediction (reloading the model) =====")
print(f"The first 10 true labels: {y_test_lab[:10].astype(int)}")
print(f"The first 10 prediction results: {np.argmax(results_reload, axis=-1)}")